In [14]:
# PRIME GENERATOR

from random import randrange, getrandbits
def is_prime(n, k = 64):
    if n == 2:
        return True
    elif n == 1:
        return False
    elif n % 2==0 and n != 2:
        return False
    
    #n - 1 = r*2^s find r and s
    r = n - 1
    s = 0
    while r % 2 == 0:
        r //= 2
        s+= 1

    for i in range(k):
        if miller_rabin(r, n, s) == False:
            return False
    else:
        return True
    

def miller_rabin(r, n, s):
     #find r, s (n-1) = r*2^s

    j = 2
    a = randrange(1, n-1)
    fermat_test =  pow(a, r, n)
    if fermat_test == 1 or fermat_test == n - 1:
        return True
    while j < s:

        x = pow(a, r*pow(2, j), n)
        if x == (1%n):
            return True
        elif x== -1:
            return False
        j += 1
    else:
        x = pow(a, r*pow(2, s-1), n)
        
        euler_test = pow(a, int((n-1)/2), n)
       
        if x == euler_test and abs(x) == 1:
            return True
        else:
            return False
        
        
        
def generate_candidate(length = 32):
    p = getrandbits(length)
    p = p | (1 << length - 1) | 1
    return p


def generate_prime(length = 32):
    result = False
    while result == False:
        candidate = generate_candidate(length)
        result = is_prime(candidate)
    return candidate 

In [20]:
# DEFs

import random as rd

def extended_gcd(a, b):
    '''Extended Euclidean Algorithm to find gcd and coefficients'''
    if b == 0:
        return a, 1, 0

    gcd, x1, y1 = extended_gcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return gcd, x, y

def gcd(p, q):
    '''returns the greatest common factor of two numbers'''
    return extended_gcd(p,q)[0]

def phi_gen(p, q):
    '''returns the totient of two PRIME numbers'''
    return (p - 1) * (q - 1)

def n_gen(p, q):
    '''returns n'''
    return p * q

def e_gen(p, q):
    '''returns e'''
    tot = phi_gen(p, q)
    for e in range(17, tot):
        if gcd(e, tot) == 1:
            break
    return e

def d_gen(p, q):
    '''returns d'''
    gcd, x, _ = extended_gcd(e_gen(p, q), phi_gen(p, q))
    return x % phi_gen(p, q)

def key_gen():
    p, q = generate_prime(1024), generate_prime(1024) # the parentheses can be left blank, but have 512 will make the prime much larger
    n = n_gen(p, q)
    e = e_gen(p, q)
    d = d_gen(p, q)
    phi = phi_gen(p, q)
    return (n, e, d)


def convert(string):
    """string --> zero-padded ASCII (as a str. as an int would cause issue: it will kill the trailing zeros)"""
    code = ''
    for char in string:
        code += f"{ord(char):03}"  # force 3 digits per char
    return code

def convert_back(decrypted_code_str):
    """zero-padded ASCII string --> original string"""
    message = ""
    # Read in chunks of 3
    for i in range(0, len(decrypted_code_str), 3):
        message += chr(int(decrypted_code_str[i:i+3]))
    return message

def encryption(message, n, e):
    '''C = m^e mod n'''
    """Return (ciphertext, length_of_ascii_str)."""
    code_str = convert(message)          
    m_len = len(code_str) # measure how many digits in total
    m_int = int(code_str) # convert to int for RSA (will result in loss of trailing zero)
    c = pow(m_int, e, n)  # encrypt
    return c, m_len


def decryption(cipher_pair, n, d):
    """Given (ciphertext, length), return the original message string."""
    c, m_len = cipher_pair
    decrypted_int = pow(c, d, n)  # standard RSA decryption
    decrypted_str = str(decrypted_int).rjust(m_len, '0') # rjust put the right amount of trailing zeros so that the str reaches an desired length (m_len)
    return convert_back(decrypted_str)

In [ ]:
# GEN KEY

n, e, d = key_gen()
print("Public Key: (n, e)", (n, e))
print("Private Key: (n, d)", (n, d))